In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Colab Notebooks/

Mounted at /content/drive/
/content/drive/My Drive/Colab Notebooks


In [322]:
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder, StandardScaler
from pyspark.ml.classification import DecisionTreeClassifier, LogisticRegression
from pyspark.mllib.regression import LinearRegressionWithSGD, LabeledPoint
from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import *
from pyspark.sql import *

# Question 1

In [347]:
sqlContext = SQLContext(sc)

df = sqlContext.read.format(source="com.databricks.spark.csv").options(header = 'true', inferSchema = 'true').load(path = '/content/drive/My Drive/train.csv')

In [ ]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



# Question 2

In [ ]:
df.show(10)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [ ]:
df.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

In [ ]:
df.groupBy('Name').count().orderBy("count", ascending = False).show(10)

+--------------------+-----+
|                Name|count|
+--------------------+-----+
|"Watt, Mrs. James...|    1|
|Young, Miss. Mari...|    1|
|Parr, Mr. William...|    1|
|Soholt, Mr. Peter...|    1|
|Goldsmith, Mrs. F...|    1|
|    Dimic, Mr. Jovan|    1|
|Harper, Mr. Henry...|    1|
|Reuchlin, Jonkhee...|    1|
|Fahlstrom, Mr. Ar...|    1|
|Hosono, Mr. Masabumi|    1|
+--------------------+-----+
only showing top 10 rows



In [ ]:
df.groupBy('PassengerId').count().orderBy("count", ascending = False).show(10)

+-----------+-----+
|PassengerId|count|
+-----------+-----+
|        148|    1|
|        463|    1|
|        471|    1|
|        496|    1|
|        833|    1|
|        243|    1|
|        392|    1|
|        540|    1|
|        623|    1|
|        737|    1|
+-----------+-----+
only showing top 10 rows



In [ ]:
df.groupBy('Sex').count().orderBy("count", ascending = False).show(10)

+------+-----+
|   Sex|count|
+------+-----+
|  male|  577|
|female|  314|
+------+-----+



In [ ]:
df.groupBy('Ticket').count().orderBy("count", ascending = False).show(10)

+------------+-----+
|      Ticket|count|
+------------+-----+
|      347082|    7|
|        1601|    7|
|    CA. 2343|    7|
|     3101295|    6|
|      347088|    6|
|     CA 2144|    6|
|S.O.C. 14879|    5|
|      382652|    5|
|        4133|    4|
|  W./C. 6608|    4|
+------------+-----+
only showing top 10 rows



In [ ]:
df.groupBy('Ticket').count().groupBy('count').count().orderBy("count", ascending = False).show(10)

+-----+-----+
|count|count|
+-----+-----+
|    7|    3|
|    6|    3|
|    5|    2|
|    4|   11|
|    3|   21|
|    2|   94|
|    1|  547|
+-----+-----+



In [ ]:
df.groupBy('Cabin').count().orderBy("count", ascending = False).show(10)

+-----------+-----+
|      Cabin|count|
+-----------+-----+
|       null|  687|
|         G6|    4|
|C23 C25 C27|    4|
|    B96 B98|    4|
|       E101|    3|
|    C22 C26|    3|
|          D|    3|
|         F2|    3|
|        F33|    3|
|        E44|    2|
+-----------+-----+
only showing top 10 rows



In [ ]:
df.groupBy('Cabin').count().groupBy('count').count().orderBy("count", ascending = False).show(10)

+-----+-----+
|count|count|
+-----+-----+
|  687|    1|
|    4|    3|
|    3|    5|
|    2|   38|
|    1|  101|
+-----+-----+



In [ ]:
df.groupBy('Embarked').count().orderBy("count", ascending = False).show(10)

+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  644|
|       C|  168|
|       Q|   77|
|    null|    2|
+--------+-----+



In [ ]:
df.groupBy('pclass').count().orderBy("count", ascending = False).show(10)

+------+-----+
|pclass|count|
+------+-----+
|     3|  491|
|     1|  216|
|     2|  184|
+------+-----+



In [348]:
df = df.drop("PassengerId","Name","Ticket","Cabin")

# Question 3

In [349]:
df = df.withColumn("SibSp",df["SibSp"].cast("double")).withColumn("Pclass",df["Pclass"].cast("double"))\
.withColumn("Parch",df["Parch"].cast("double")).withColumn("Survived",df["Survived"].cast("double"))

In [350]:
df.printSchema()

root
 |-- Survived: double (nullable = true)
 |-- Pclass: double (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: double (nullable = true)
 |-- Parch: double (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)



In [351]:
df = df.withColumn("Age_M",df.Age.isNull())
df.show(10)

+--------+------+------+----+-----+-----+-------+--------+-----+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|Age_M|
+--------+------+------+----+-----+-----+-------+--------+-----+
|     0.0|   3.0|  male|22.0|  1.0|  0.0|   7.25|       S|false|
|     1.0|   1.0|female|38.0|  1.0|  0.0|71.2833|       C|false|
|     1.0|   3.0|female|26.0|  0.0|  0.0|  7.925|       S|false|
|     1.0|   1.0|female|35.0|  1.0|  0.0|   53.1|       S|false|
|     0.0|   3.0|  male|35.0|  0.0|  0.0|   8.05|       S|false|
|     0.0|   3.0|  male|null|  0.0|  0.0| 8.4583|       Q| true|
|     0.0|   1.0|  male|54.0|  0.0|  0.0|51.8625|       S|false|
|     0.0|   3.0|  male| 2.0|  3.0|  1.0| 21.075|       S|false|
|     1.0|   3.0|female|27.0|  0.0|  2.0|11.1333|       S|false|
|     1.0|   2.0|female|14.0|  1.0|  0.0|30.0708|       C|false|
+--------+------+------+----+-----+-----+-------+--------+-----+
only showing top 10 rows



In [352]:
df = df.withColumn("Age_M",df["Age_M"].cast("double"))

In [353]:
mean_age=df.select(mean(df.Age)).collect()[0][0]
print(mean_age)

29.69911764705882


In [354]:
df = df.na.fill(mean_age,subset=['Age'])

In [355]:
df.show(10)

+--------+------+------+-----------------+-----+-----+-------+--------+-----+
|Survived|Pclass|   Sex|              Age|SibSp|Parch|   Fare|Embarked|Age_M|
+--------+------+------+-----------------+-----+-----+-------+--------+-----+
|     0.0|   3.0|  male|             22.0|  1.0|  0.0|   7.25|       S|  0.0|
|     1.0|   1.0|female|             38.0|  1.0|  0.0|71.2833|       C|  0.0|
|     1.0|   3.0|female|             26.0|  0.0|  0.0|  7.925|       S|  0.0|
|     1.0|   1.0|female|             35.0|  1.0|  0.0|   53.1|       S|  0.0|
|     0.0|   3.0|  male|             35.0|  0.0|  0.0|   8.05|       S|  0.0|
|     0.0|   3.0|  male|29.69911764705882|  0.0|  0.0| 8.4583|       Q|  1.0|
|     0.0|   1.0|  male|             54.0|  0.0|  0.0|51.8625|       S|  0.0|
|     0.0|   3.0|  male|              2.0|  3.0|  1.0| 21.075|       S|  0.0|
|     1.0|   3.0|female|             27.0|  0.0|  2.0|11.1333|       S|  0.0|
|     1.0|   2.0|female|             14.0|  1.0|  0.0|30.0708|  

In [356]:
df = df.na.drop()

In [357]:
df.show(10)

+--------+------+------+-----------------+-----+-----+-------+--------+-----+
|Survived|Pclass|   Sex|              Age|SibSp|Parch|   Fare|Embarked|Age_M|
+--------+------+------+-----------------+-----+-----+-------+--------+-----+
|     0.0|   3.0|  male|             22.0|  1.0|  0.0|   7.25|       S|  0.0|
|     1.0|   1.0|female|             38.0|  1.0|  0.0|71.2833|       C|  0.0|
|     1.0|   3.0|female|             26.0|  0.0|  0.0|  7.925|       S|  0.0|
|     1.0|   1.0|female|             35.0|  1.0|  0.0|   53.1|       S|  0.0|
|     0.0|   3.0|  male|             35.0|  0.0|  0.0|   8.05|       S|  0.0|
|     0.0|   3.0|  male|29.69911764705882|  0.0|  0.0| 8.4583|       Q|  1.0|
|     0.0|   1.0|  male|             54.0|  0.0|  0.0|51.8625|       S|  0.0|
|     0.0|   3.0|  male|              2.0|  3.0|  1.0| 21.075|       S|  0.0|
|     1.0|   3.0|female|             27.0|  0.0|  2.0|11.1333|       S|  0.0|
|     1.0|   2.0|female|             14.0|  1.0|  0.0|30.0708|  

In [358]:
df.describe().show()

+-------+-------------------+------------------+------+------------------+------------------+-------------------+-----------------+--------+-------------------+
|summary|           Survived|            Pclass|   Sex|               Age|             SibSp|              Parch|             Fare|Embarked|              Age_M|
+-------+-------------------+------------------+------+------------------+------------------+-------------------+-----------------+--------+-------------------+
|  count|                889|               889|   889|               889|               889|                889|              889|     889|                889|
|   mean|0.38245219347581555|2.3115860517435323|  null|29.653446370674192|0.5241844769403825|0.38245219347581555|32.09668087739029|    null|0.19910011248593926|
| stddev|0.48625968831477334|0.8346997785705753|  null|12.968366309252314| 1.103704875596923| 0.8067607445174785|49.69750431670795|    null| 0.3995482811002537|
|    min|                0.0|     

# Question 4

In [359]:
sex_indexer = StringIndexer(inputCol="Sex",outputCol="Sex_ix")
embarked_indexer = StringIndexer(inputCol="Embarked",outputCol="Embarked_ix")

In [360]:
sex_encoder = OneHotEncoder(inputCol="Sex_ix",outputCol="Sex_oh")
embarked_encoder = OneHotEncoder(inputCol="Embarked_ix",outputCol="Embarked_oh")

# Question 5

In [362]:
x_cols = ["Pclass", "Sex_oh", "Age", "Age_M", "SibSp", "Parch", "Fare", "Embarked_oh"]

va = VectorAssembler(inputCols = x_cols, outputCol = 'features')

# Question 6

In [381]:
lr = LogisticRegression(maxIter=10,featuresCol = "scaled", labelCol = "Survived")

# Question 7

In [364]:
ss = StandardScaler(withMean=True, withStd = True, inputCol = "features", outputCol = "scaled")

In [382]:
steps = [sex_indexer, embarked_indexer, sex_encoder, embarked_encoder, va, ss, lr]

In [383]:
pl = Pipeline(stages=steps)

# Question 8

In [368]:
from pyspark.ml.tuning import TrainValidationSplit

train, test = df.randomSplit([0.7, 0.3], seed=666)

In [369]:
train.count()

631

In [370]:
test.count()

258

In [371]:
train.printSchema()

root
 |-- Survived: double (nullable = true)
 |-- Pclass: double (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = false)
 |-- SibSp: double (nullable = true)
 |-- Parch: double (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Age_M: double (nullable = false)



# Question 9

In [384]:
plmodel = pl.fit(train)

In [385]:
predictions = plmodel.transform(test)

In [388]:
x_cols = ["Pclass", "Sex_oh", "Age", "Age_M", "SibSp", "Parch", "Fare", "Embarked_oh"]

x_cols.append("Embarked_oh2")
x_cols.append("intercept")

coefficients = list(plmodel.stages[-1].coefficients)
coefficients.append(plmodel.stages[-1].intercept)

In [389]:
import pandas as pd

pd.DataFrame([coefficients], columns = x_cols)

,Pclass,Sex_oh,Age,Age_M,SibSp,Parch,Fare,Embarked_oh,Embarked_oh2,intercept
0,-0.896848,-1.328635,-0.413685,-0.080899,-0.438067,-0.116797,0.026506,-0.233985,-0.101337,-0.623776


# Question 10

In [390]:
predictions.show(5,truncate=False)

+--------+------+----+-----------------+-----+-----+------+--------+-----+------+-----------+-------------+-------------+------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------+----------------------------------------+----------+
|Survived|Pclass|Sex |Age              |SibSp|Parch|Fare  |Embarked|Age_M|Sex_ix|Embarked_ix|Sex_oh       |Embarked_oh  |features                                              |scaled                                                                                                                                                                             |rawPrediction                             |probability                             |prediction|
+--------+------+----+-----------------+-----+-----+------+--------+-----+------+-----------+---

In [391]:
print("AUC outsample : %s" % plmodel.stages[-1].summary.areaUnderROC)

AUC outsample : 0.855969551282051


In [392]:
from sklearn.metrics import roc_auc_score

cols = ['probability', 'Survived']
evalue = predictions.select(cols)

evalue = evalue.select("*").toPandas()
evalue['probability'] = evalue['probability'].apply(lambda x: pd.Series(x.toArray()))[1]

print("AUC outsample : %s" % roc_auc_score(evalue["Survived"], evalue["probability"]))

AUC outsample : 0.8546106223525578
